# <center>Store Sales Time Series Forecasting</center>

## Business Understanding
### Introduction
The retail indstry is characterized by seasonal flunctuations, consumer trends, and promotions. Acurate sales forecasting is critical to optimize invvenntor levels, minimize stockouts, and reduce excess inventory costs.

### Business Objective
The object of this projects topredict unit store sales on data from Corporation Favorria, a large Ecuadorian-based grocery retailer. We will be building a model that accurately preddicts the unit sales for thousands of items sold at different Favorita stores. insights from this project will be beneficial for Inventory Management, Demand Planning, Resource Allocation, Promotion planning, Operation Efficiency, Financial Planning, Profitability, Cstomer satisfaction, new product intrroduction and so
### Data Reqirements
For this project we will need informaation features that can affect sales eg, Historical Sales Data, product Informattion, Promotional information, Store Data, External Factors Data and so on.
and# itHub

#


# Hypothesis
### Questions

1. Is the train dataset complete (has all the required dates)?

2. Which dates have the lowest and highest sales for each year?

3. Did the earthquake impact sales?

4. Are certain groups of stores selling more products? (Cluster, city, state, type)

5. Are sales affected by promotions, oil prices and holidays?

6. What analysis can we get from the date and its extractable features?

7. What is the difference between RMSLE, RMSE, MSE (or why is the MAE which greatly affected supermarket sales for several weeks after the earthquake.

### Install required packages


In [1]:
#Libraries for sql
import pyodbc 
from dotenv import dotenv_values #import the dotenv_values function from the dotenv package
import warnings 
warnings.filterwarnings('ignore')

#libraries for handling data
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Create a connection by accessing connection string with defined environment variables


In [2]:
# Load environment variables from .env file into a dictionary
environment_variables = dotenv_values('.env')


In [3]:
# Get the values for the credentials you set in the '.env' file
database = environment_variables.get("DATABASE")
server = environment_variables.get("SERVER")
username = environment_variables.get("USERNAME")
password = environment_variables.get("PASSWORD")

connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"


In [4]:
# Use the connect method of the pyodbc library and pass in the connection string.
connection = pyodbc.connect(connection_string)

In [5]:
# Now the sql query to get the data is what what you see below. 
query = "Select * from dbo.oil"
query1 = "Select * from dbo.holidays_events"
query2 = "Select * from dbo.stores"

In [6]:
oil = pd.read_sql(query, connection)
holidays_events = pd.read_sql(query1, connection)
stores = pd.read_sql(query2, connection)

In [34]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.139999
2,2013-01-03,92.970001
3,2013-01-04,93.120003
4,2013-01-07,93.199997


In [8]:
oil.shape

(1218, 2)

In [9]:
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [10]:
holidays_events.shape

(350, 6)

In [19]:
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [20]:
stores.shape

(54, 5)

In [12]:
#sample = pd.read_csv(r"C:\Users\HP\Desktop\TSA_LP3\Dataset\sample_submission.csv")
test = pd.read_csv(r"C:\Users\HP\Desktop\TSA_LP3\Dataset\test.csv")
train = pd.read_csv(r"C:\Users\HP\Desktop\TSA_LP3\Dataset\train.csv")
tans = pd.read_csv(r"C:\Users\HP\Desktop\TSA_LP3\Dataset\transactions.csv")

In [13]:
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [14]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [15]:
train.tail()

,id,date,store_nbr,family,sales,onpromotion
3000883,3000883,2017-08-15,9,POULTRY,438.133,0
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.553,1
3000885,3000885,2017-08-15,9,PRODUCE,2419.729,148
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8
3000887,3000887,2017-08-15,9,SEAFOOD,16.000,0


In [16]:
train.shape

(3000888, 6)

In [26]:
tans.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [32]:
tans['store_nbr'].nunique()

54

In [38]:
tans['date'].nunique()

1682

In [31]:
stores['store_nbr'].nunique()

54

In [35]:
oil['date'].nunique()

1218

In [39]:
train['store_nbr'].nunique()

54

In [37]:
holidays_events['date'].nunique()

312

In [24]:
merged_df = tans.merge(stores, on='store_nbr', how='left')

# Fill missing state values in the merged DataFrame
merged_df[['city', 'type', 'cluster', 'state']].fillna(method='ffill', inplace=True)

merged_df.head()

,date,store_nbr,transactions,city,state,type,cluster
0,2013-01-01,25,770,Salinas,Santa Elena,D,1
1,2013-01-02,1,2111,Quito,Pichincha,D,13
2,2013-01-02,2,2358,Quito,Pichincha,D,13
3,2013-01-02,3,3487,Quito,Pichincha,D,8
4,2013-01-02,4,1922,Quito,Pichincha,D,9


In [41]:
merged_df1 = merged_df.merge(train, on='store_nbr', how='left')

# Fill missing state values in the merged DataFrame
merged_df1[['id', 'store_nbr', 'family', 'sales', 'onpromotion']].fillna(method='ffill', inplace=True)

merged_df1.head()

MemoryError: Unable to allocate 34.6 GiB for an array with shape (4639595136,) and data type int64

In [40]:
train.columns

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion'], dtype='object')

In [18]:
tans.shape

(83488, 3)

## Data Understanding

### Data Collection
Data for this project will be collected from 3 places, a database, OneDrive and GitHub

### Data Description
The training data includes dates, store, and product information, whether that item was being promoted, as well as the sales numbers. Additional files include supplementary information that may be useful in building your models

**File Descriptions and Data Field Information**

train.csv

- The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.

- **store_nbr** identifies the store at which the products are sold.

- **family** identifies the type of product sold.

- **sales** gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).

- **onpromotion** gives the total number of items in a product family that were being promoted at a store at a given date.

test.csv

- The test data, having the same features as the training data. You will predict the target sales for the dates in this file.

- The dates in the test data are for the 15 days after the last date in the training data.

transaction.csv

- Contains date, store_nbr and transaction made on that specific date.

sample_submission.csv

- A sample submission file in the correct format.

stores.csv

- Store metadata, including city,state, type, and cluster.

- cluster is a grouping of similar stores.

oil.csv

- **Daily oil price** which includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and its economical health is highly vulnerable to shocks in oil prices.)

holidays_events.csv

- Holidays and Events, with metadata

Additional holidays are days added, a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

Additional Notes

- Wages in the public sector are paid every two weeks on the 15th and on the last day of the month. Supermarket sales could be affected by this.

- A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.